# Notebook for the paper implementation
## Unfortunately takes a lot of time to run without many GPUs

In [12]:
import jax

In [13]:
from datasets import load_cifar_dataset, PoisonedDataset, LabelPoisoner,PixelPoisoner

# Load the CIFAR-10 dataset
train_dataset = load_cifar_dataset(train=True)
test_dataset = load_cifar_dataset(train=False)




Files already downloaded and verified
Files already downloaded and verified


In [14]:
len(train_dataset), len(test_dataset)

(50000, 10000)

In [15]:
import numpy as np

In [16]:


# Extract images and labels
train_images = np.array([sample[0] for sample in train_dataset])
train_labels = np.array([sample[1] for sample in train_dataset])
test_images = np.array([sample[0] for sample in test_dataset])
test_labels = np.array([sample[1] for sample in test_dataset])

# Stack images and labels separately
all_images = np.vstack([train_images, test_images])
all_labels = np.hstack([train_labels, test_labels])

# Save to .npz format for better organization
np.savez('output/X-94-1xs-perm', images=all_images, labels=all_labels)

In [17]:
# Initialize a poisoner for specific labels if needed
pix=PixelPoisoner()
poisoner = LabelPoisoner(pix, target_label=1)
poisoned_train_dataset = PoisonedDataset(train_dataset, poisoner, label=0, eps=100)

In [18]:
from ntk_cifar import kernel_fit, x_train_c, y_train_c, x_test_c, y_test_c
from models import WideResnet,SimpleCNN,ConvNeXt,Small_fc_network

In [19]:
#x_train_c, y_train_c = x_train_c[:90], y_train_c[:90]

In [20]:


# Initialize the kernel function
#_, _, kernel_fn = WideResnet(block_size=4, k=5, num_classes=1)
_, _, kernel_fn =Small_fc_network()

# Fit the kernel to the training data
model, g_dd, predictor = kernel_fit(kernel_fn, x_train_c, y_train_c)


MemoryError: Unable to allocate 400. MiB for an array with shape (1, 10, 10, 32, 32, 32, 32) and data type float32

In [ ]:
from train_fast import iter, lr, tx, params, opt_state
import jax

# Initialize JAX key and model parameters
key = jax.random.PRNGKey(0)
num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    (params, opt_state, key), metrics = iter(epoch, params, opt_state, key)
    avg_loss, avg_acc, avg_tloss, avg_tacc, avg_ploss, avg_pacc, p_pred = metrics
    print(f"Epoch {epoch}: Loss={avg_loss:.4f}, Accuracy={avg_acc:.4f}")


KeyError: 'slice(0, 1000, None) is not a file in the archive'

In [ ]:
from util import compute_all_reps, clf_eval

# Evaluate on the test set
test_acc, test_loss = clf_eval(model, test_dataset)
print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")

# Extract representations from specific layers
layer_reps = compute_all_reps(model, train_dataset, layers=[0, 2, 4])
print("Layer representations extracted.")


In [ ]:
import wandb

wandb.init(project="ntk-backdoor", entity="your_entity")
wandb.log({"train_loss": avg_loss, "train_acc": avg_acc})
